In [1]:
# Declaration

import pandas as pd
import numpy as np
from enum import Enum
from enum import IntEnum

class Disease(IntEnum):
    CHD = 53741008
    DIABETES = 44054006
    HYPERTENSION = 38341003
    MI = 22298006
    STROKE = 230690007

class DrugDiabetes(IntEnum):
    METFORMIN = 860975
    INSULIN = 106892
    GLP1 = 897122
    SGLT2 = 1373463

class Observation(Enum):
    BH = '8302-2'
    BW = '29463-7'
    BMI = '39156-5'
    DIASTOLIC = '8462-4'
    GLUCOSE = '2339-0'
    HBA1C = '4548-4'
    SYSTOLIC = '8480-6'


## LOAD & CLEANING DATA PER SET

source : set(setnumber)\_raw/
dest   : set(setnumber)/

In [201]:
cleaning_set = 'set9'

# loading data
def load_data(type):
    data = pd.read_csv(cleaning_set + '/' + type + '.csv')
    return data

In [141]:
# cleaning - patients
def cleaning_patients():
    # patients
    pat = pd.read_csv(cleaning_set + '_raw/patients.csv', sep=';', usecols=range(17))
    pat = pat.drop(
        ['SSN', 'DRIVERS', 'PASSPORT', 'PREFIX', 'FIRST', 'LAST', 'SUFFIX', 'MARITAL', 'MAIDEN', 'BIRTHPLACE', 'ADDRESS' ], 
        axis=1
    )

    # clean data due to malformed csv
    race_list = ['asian', 'white', 'black', 'hispanic']
    pat['RACE'].replace(pat[~pat['RACE'].isin(race_list)]['RACE'].array, np.nan, inplace=True)
    pat = pat[pat['RACE'].notna()]

    ethnicity_list = ['african', 'american', 'american_indian', 'asian_indian', 'central_american', 'chinese', 'dominican', 'english', 'french', 'french_canadian', 'german', 'irish', 'italian', 'mexican', 'polish', 'portuguese', 'puerto_rican', 'russian', 'scottish', 'swedish', 'west_indian']
    pat['ETHNICITY'].replace(pat[~pat['ETHNICITY'].isin(ethnicity_list)]['ETHNICITY'].array, np.nan, inplace=True)

    gender_list = ['F', 'M']
    pat['GENDER'].replace(pat[~pat['GENDER'].isin(gender_list)]['GENDER'].array, np.nan, inplace=True)
    pat = pat[pat['GENDER'].notna()]

    # save
    pat.to_csv(cleaning_set + '/patients.csv', index=False)

cleaning_patients()

In [142]:
# cleaning - medications

def cleaning_medications():
    # remove all medications without patient data
    pat = load_data('patients')

    med = pd.read_csv(cleaning_set + '_raw/medications.csv')        
    med = med[med['PATIENT'].isin(pat['ID'])]
    med = med[~med['REASONCODE'].isna()]
    med = med.drop_duplicates()
    med['REASONCODE'] = med['REASONCODE'].astype('int')

    #med['STOP'] = med['STOP'].fillna('9999-01-01')


    # save
    med.to_csv(cleaning_set + '/medications.csv', index=False)
    display(med)

cleaning_medications()

,START,STOP,PATIENT,ENCOUNTER,CODE,DESCRIPTION,REASONCODE,REASONDESCRIPTION
2,2010-05-21,2010-05-31,0e423dfd-9268-460d-8f0d-5f7d7f2606e8,edd87211-8c9b-4619-b575-77500dfc1db2,608680,Acetaminophen 160 MG,10509002,Acute bronchitis (disorder)
8,2012-01-14,NaN,0e423dfd-9268-460d-8f0d-5f7d7f2606e8,bee13a5f-ccfc-4141-81be-73cdd84e74bd,895994,120 ACTUAT Fluticasone propionate 0.044 MG/ACT...,233678006,Childhood asthma
9,2012-01-14,NaN,0e423dfd-9268-460d-8f0d-5f7d7f2606e8,bee13a5f-ccfc-4141-81be-73cdd84e74bd,745679,200 ACTUAT Albuterol 0.09 MG/ACTUAT Metered Do...,233678006,Childhood asthma
21,2010-12-08,2010-12-23,8081a1c2-213d-4bd4-a8f2-c1127faf67e3,19c6fc3a-0c57-40c5-9871-f79e4991ef09,608680,Acetaminophen 160 MG,10509002,Acute bronchitis (disorder)
34,1997-10-02,NaN,033083d6-d4aa-40fb-adb2-f560b8ffd3b8,6d2b02ee-9043-4a8e-b62c-a046b0c392b5,834060,Penicillin V Potassium 250 MG,43878008,Streptococcal sore throat (disorder)
...,...,...,...,...,...,...,...,...
399775,1977-07-25,NaN,1f75d47b-9e62-4d7f-a046-120a946bf0a3,f625d70f-687d-4d0d-974f-720908dab98a,834101,Penicillin V Potassium 500 MG,43878008,Streptococcal sore throat (disorder)
399776,1988-06-07,NaN,1f75d47b-9e62-4d7f-a046-120a946bf0a3,f625d70f-687d-4d0d-974f-720908dab98a,834101,Penicillin V Potassium 500 MG,43878008,Streptococcal sore throat (disorder)
399778,1948-11-06,NaN,78e5e18a-7bbd-4a90-9627-065049224001,d46ffc3b-766b-4603-82e0-65865c8eecee,834060,Penicillin V Potassium 250 MG,43878008,Streptococcal sore throat (disorder)
399779,1985-02-11,NaN,78e5e18a-7bbd-4a90-9627-065049224001,d46ffc3b-766b-4603-82e0-65865c8eecee,860975,24 HR Metformin hydrochloride 500 MG Extended ...,44054006,Diabetes


In [143]:
# cleaning - encounters

def cleaning_encounters():
    # remove all encounters without patient data
    pat = load_data('patients')

    enc = pd.read_csv(cleaning_set + '_raw/encounters.csv')
    # most of it is null
    enc = enc.drop(
        ['DESCRIPTION', 'REASONDESCRIPTION'], 
        axis=1
    )
    enc = enc[enc['PATIENT'].isin(pat['ID'])]

    # save
    enc.to_csv(cleaning_set + '/encounters.csv', index=False)
    
cleaning_encounters()

In [144]:
# cleaning - observations

def cleaning_observations():
    # remove all encounters without patient data
    pat = load_data('patients')
    obs = pd.read_csv(cleaning_set + '_raw/observations.csv')
    obs = obs[obs['PATIENT'].isin(pat['ID'])]
    obs = obs[~obs['VALUE'].isna()]
    obs = obs.loc[
        (obs.CODE==Observation.BH.value)
        | (obs.CODE==Observation.BW.value)
        | (obs.CODE==Observation.BMI.value)
        | (obs.CODE==Observation.DIASTOLIC.value)
        | (obs.CODE==Observation.GLUCOSE.value)
        | (obs.CODE==Observation.HBA1C.value)
        | (obs.CODE==Observation.SYSTOLIC.value)
    ]
    obs = obs.drop_duplicates()
    # save
    obs.to_csv(cleaning_set + '/observations.csv', index=False)

cleaning_observations()

In [202]:
# cleaning - conditions

def cleaning_conditions():
    pat = load_data('patients')
    con = pd.read_csv(cleaning_set + '_raw/conditions.csv')
    con = con[con['PATIENT'].isin(pat['ID'])]
    con = con[
        (con['CODE'] == Disease.DIABETES)
        | (con['CODE'] == Disease.HYPERTENSION)
        | (con['CODE'] == Disease.CHD)
    ]
    con = con.drop_duplicates()

    #save
    con.to_csv(cleaning_set + '/conditions.csv', index=False)

cleaning_conditions()


## CONCAT ALL DATASET FROM ALL SET PER TYPE

source : set1/, set2/, set3/, ...

target : set_full/

files :
- patients.csv : ID, BIRTHDATE, DEATHDATE, RACE, ETHNICITY, GENDER
- medications.csv  : START, STOP, PATIENT, ENCOUNTER, CODE, DESCRIPTION, REASONCODE, REASONDESCRIPTION
- encounters.csv   : ID, DATE, PATIENT, CODE, REASONCODE
- observations.csv : DATE, PATIENT, ENCOUNTER, CODE, DESCRIPTION, VALUE, UNITS

In [203]:
def concat_set(type, max_set):
    result = pd.read_csv('set1/' + type + '.csv')
    for i in range(2, max_set+1):
        temp = pd.read_csv('set' + str(i) + '/' + type + '.csv')
        result = result.append(temp).reset_index(drop=True)
    result.to_csv('set_full/' + type + '.csv', index=False)
    return result

# [patients | medications | observations | encounters | conditions]
data = concat_set('conditions', 9)
#data

## DIABETES


## Characteristic of the medication data

Fact :
- All metformin do not have STOP date
- There is no record of someone receive > 1 record of metformin

Set used : 1-3
Medication durations per record (in days)
- METFORMIN
    - mean : -
    - median : -
- SGLT2
    - mean : 354.24 (~ 1 year)
    - median : 352 (~ 1 year)
- GLP1
    - mean : 992.80 (~ 3 year)
    - median : 659 (~ 2 years)
- INSULIN
    - count : 396
    - mean : 1923.86 (~ 5 years)
    - median : 631 (~ 2 years)

The data above shows that the duration is not a date when the patient get drugs from prescription. It is possible that during periode between start and date, the patient has more than one prescription.

Base on that fact, it is reasonable to make assumptions:
- those who receive metformin as their first medication will always use it forever because metformin has no end date
- if there is another medication found after metformin, then it will be a double or triple medications combined with it (this is relevant with NICE guideline)

Question :
- for the intensification assumption, can it apply for insulin?
  according to NICE insulin is a single medication and not to be used together with other medication


In [146]:
# load medications
medications = pd.read_csv('set_full/medications.csv')
# diabetes only
medications = medications[medications['REASONCODE'] == Disease.DIABETES].reset_index(drop=True)
# drop unused columns
medications = medications.drop(['REASONCODE', 'REASONDESCRIPTION'], axis=1)

medications['START'] = pd.to_datetime(medications['START'], format='%Y-%m-%d')
medications['STOP'] = pd.to_datetime(medications['STOP'], format='%Y-%m-%d')

# get medication duration
medications['DURATION'] = (medications['STOP'] - medications['START']).dt.days
# medications = medications[medications['DURATION'] > 0]

In [152]:
# calculate medical duration per drugs
metformin = medications[medications['CODE'] == DrugDiabetes.METFORMIN]
#metformin.describe()

In [148]:
# calculate medical duration per drugs
sglt2 = medications[medications['CODE'] == DrugDiabetes.SGLT2]
sglt2.describe()

,CODE,DURATION
count,3979.0,3287.000000
mean,1373463.0,354.740189
std,0.0,107.760363
min,1373463.0,3.000000
25%,1373463.0,326.000000
50%,1373463.0,351.000000
75%,1373463.0,369.000000
max,1373463.0,1297.000000


In [149]:
# calculate medical duration per drugs
glp1 = medications[medications['CODE'] == DrugDiabetes.GLP1]
glp1.describe()

,CODE,DURATION
count,15398.0,10646.000000
mean,897122.0,941.435375
std,0.0,1449.918196
min,897122.0,0.000000
25%,897122.0,366.000000
50%,897122.0,651.000000
75%,897122.0,846.000000
max,897122.0,21440.000000


In [150]:
# calculate medical duration per drugs
insulin = medications[medications['CODE'] == DrugDiabetes.INSULIN]
insulin.describe()

,CODE,DURATION
count,23028.0,1164.000000
mean,106892.0,1887.907216
std,0.0,2247.712238
min,106892.0,0.000000
25%,106892.0,287.750000
50%,106892.0,534.000000
75%,106892.0,3622.750000
max,106892.0,10378.000000


In [151]:
# There is no record of someone get more than one metformin

# fill medication stop with duration median (per medication)
met = medications[medications['CODE'] == DrugDiabetes.METFORMIN].reset_index(drop=True)
met['MEDNUM'] = met.groupby(by=['PATIENT']).cumcount() + 1
#met.set_index(['PATIENT', 'ENCOUNTER'])
met[met.MEDNUM > 1]

,START,STOP,PATIENT,ENCOUNTER,CODE,DESCRIPTION,DURATION,MEDNUM


### 1. Medications

Transform medications from the same date into separated columns

- file : "diabetes/m.csv"
- fields : PATIENT, ENCOUNTER, MEDSTART, INSULIN, METFORMIN, GLP1, SGLT2, COMBINATION, MEDCLASS

Step by step :
- One hot encoding for medication
- Combine all medications on the same date
- If there is no stop date, then the treatment data on line [i] will continue to run on line [i + 1], but if there is a stop date, it will not carry over to the next medication
- So if there is a new drugs used in the next record, it will come as an intensification of the treatment

In [153]:
# load medications
medications = pd.read_csv('set_full/medications.csv')

# diabetes only
medications = medications[medications['REASONCODE'] == Disease.DIABETES].reset_index(drop=True)

# drop unused columns
medications = medications.drop(['DESCRIPTION', 'REASONCODE', 'REASONDESCRIPTION'], axis=1)

# create multiple indexes
medications = medications.set_index(['PATIENT', 'START'])

# convert medications data to columns (one hot encoding)
medications = pd.get_dummies(data = medications, columns=['CODE'])

# sort based on date
medications.sort_index(inplace=True)

medications = medications.reset_index()

# Rename drugs columns
for d in DrugDiabetes:
    medications.rename(
        columns={            
            'CODE_' + str(d.value) : d.name
        }, inplace=True
    )

medications.head(5)

,PATIENT,START,STOP,ENCOUNTER,INSULIN,METFORMIN,GLP1,SGLT2
0,000177c6-f76b-432b-9493-5a88bc9fb6bd,1957-02-27,NaN,351e39d2-52a7-4ce7-928b-5920d9761787,0,1,0,0
1,000177c6-f76b-432b-9493-5a88bc9fb6bd,1957-02-27,1994-02-21,351e39d2-52a7-4ce7-928b-5920d9761787,0,0,1,0
2,000177c6-f76b-432b-9493-5a88bc9fb6bd,1995-03-13,1996-05-07,351e39d2-52a7-4ce7-928b-5920d9761787,0,0,1,0
3,000177c6-f76b-432b-9493-5a88bc9fb6bd,1997-03-28,NaN,351e39d2-52a7-4ce7-928b-5920d9761787,0,0,1,0
4,000177c6-f76b-432b-9493-5a88bc9fb6bd,1997-03-28,NaN,351e39d2-52a7-4ce7-928b-5920d9761787,1,0,0,0


In [ ]:
#medications = medications[medications['PATIENT'].isin(['018abf0e-6843-44c7-8003-5403631d8a53', '09ce20b5-1c49-4549-af2b-f2fc6399f6a6'])]

In [154]:
result = medications.groupby(by=['PATIENT','START','ENCOUNTER']).agg(['sum'])
result.columns = result.columns.droplevel(1)
result

INSULIN  \
PATIENT                              START      ENCOUNTER                                       
000177c6-f76b-432b-9493-5a88bc9fb6bd 1957-02-27 351e39d2-52a7-4ce7-928b-5920d9761787        0   
                                     1995-03-13 351e39d2-52a7-4ce7-928b-5920d9761787        0   
                                     1997-03-28 351e39d2-52a7-4ce7-928b-5920d9761787        1   
00018e41-aba7-4cb3-aeed-fd2d7bcbc6a7 1964-07-10 7f89d06b-9fcf-432c-aa18-defed8f07c16        0   
                                     1970-02-09 7f89d06b-9fcf-432c-aa18-defed8f07c16        1   
...                                                                                       ...   
ffff216a-4273-4843-b9f3-9f81ace31098 1976-05-23 ab5c59ab-45da-477b-bed9-30cf61c289a6        1   
ffff24a2-d219-4aca-aab4-2e45a01632af 1995-09-27 dd6b92f3-0b9f-449b-8dcb-83e137807ede        1   
ffff2e14-b1e8-4a73-a79b-90dff97c8116 2016-02-29 03817b40-1bcd-4f49-93c6-2e6dad36ae91        0   
                                     2016-09-03 5c659024-a8c0-4d14-8118-920fed2ff618        1   
ffff42b6-ebed-4835-ba85-a56c2ffc0774 2010-12-04 454ec8c2-48e4-4f7c-a258-fbc2b3201ade        0   

                                                                                      METFORMIN  \
PATIENT                              START      ENCOUNTER                                         
000177c6-f76b-432b-9493-5a88bc9fb6bd 1957-02-27 351e39d2-52a7-4ce7-928b-5920d9761787          1   
                                     1995-03-13 351e39d2-52a7-4ce7-928b-5920d9761787          0   
                                     1997-03-28 351e39d2-52a7-4ce7-928b-5920d9761787          0   
00018e41-aba7-4cb3-aeed-fd2d7bcbc6a7 1964-07-10 7f89d06b-9fcf-432c-aa18-defed8f07c16          1   
                                     1970-02-09 7f89d06b-9fcf-432c-aa18-defed8f07c16          0   
...                                                                                         ...   
ffff216a-4273-4843-b9f3-9f81ace31098 1976-05-23 ab5c59ab-45da-477b-bed9-30cf61c289a6          0   
ffff24a2-d219-4aca-aab4-2e45a01632af 1995-09-27 dd6b92f3-0b9f-449b-8dcb-83e137807ede          0   
ffff2e14-b1e8-4a73-a79b-90dff97c8116 2016-02-29 03817b40-1bcd-4f49-93c6-2e6dad36ae91          1   
                                     2016-09-03 5c659024-a8c0-4d14-8118-920fed2ff618          0   
ffff42b6-ebed-4835-ba85-a56c2ffc0774 2010-12-04 454ec8c2-48e4-4f7c-a258-fbc2b3201ade          1   

                                                                                      GLP1  \
PATIENT                              START      ENCOUNTER                                    
000177c6-f76b-432b-9493-5a88bc9fb6bd 1957-02-27 351e39d2-52a7-4ce7-928b-5920d9761787     1   
                                     1995-03-13 351e39d2-52a7-4ce7-928b-5920d9761787     1   
                                     1997-03-28 351e39d2-52a7-4ce7-928b-5920d9761787     1   
00018e41-aba7-4cb3-aeed-fd2d7bcbc6a7 1964-07-10 7f89d06b-9fcf-432c-aa18-defed8f07c16     0   
                                     1970-02-09 7f89d06b-9fcf-432c-aa18-defed8f07c16     0   
...                                                                                    ...   
ffff216a-4273-4843-b9f3-9f81ace31098 1976-05-23 ab5c59ab-45da-477b-bed9-30cf61c289a6     0   
ffff24a2-d219-4aca-aab4-2e45a01632af 1995-09-27 dd6b92f3-0b9f-449b-8dcb-83e137807ede     0   
ffff2e14-b1e8-4a73-a79b-90dff97c8116 2016-02-29 03817b40-1bcd-4f49-93c6-2e6dad36ae91     0   
                                     2016-09-03 5c659024-a8c0-4d14-8118-920fed2ff618     0   
ffff42b6-ebed-4835-ba85-a56c2ffc0774 2010-12-04 454ec8c2-48e4-4f7c-a258-fbc2b3201ade     0   

                                                                                      SGLT2  
PATIENT                              START      ENCOUNTER                                    
000177c6-f76b-432b-9493-5a88bc9fb6bd 1957-02-27 351e39d2-52a7-4ce7-928b-5920d9761787      0  
                               

In [155]:
medications['STOP'].fillna('-', inplace=True)

prev_patient = ''
grouped = medications.groupby(['PATIENT', 'START', 'ENCOUNTER'])

for index, group in grouped:
    patient = index[0]
    if patient != prev_patient:
        prev_patient = patient
        carry_insulin = False
        carry_metformin = False
        carry_glp1 = False
        carry_sglt2 = False

    temp = group.reset_index(drop=True)

    # cari carry sebelum aggregate
    for i in range(0, len(temp)):
        if (temp.loc[i]['INSULIN'] == 1):
            carry_insulin = (temp.loc[i]['STOP'] == '-')
        elif (temp.loc[i]['METFORMIN'] == 1):
            carry_metformin = (temp.loc[i]['STOP'] == '-')
        elif temp.loc[i]['GLP1'] == 1:
            carry_glp1 = (temp.loc[i]['STOP'] == '-')
        elif temp.loc[i]['SGLT2'] == 1:
            carry_sglt2 = (temp.loc[i]['STOP'] == '-')
    
    if carry_insulin:
        result.at[index, 'INSULIN'] = 1

    if carry_metformin:
        result.at[index, 'METFORMIN'] = 1

    if carry_glp1:
        result.at[index, 'GLP1'] = 1

    if carry_sglt2:
        result.at[index, 'SGLT2'] = 1

#result = result.reset_index()

In [156]:
# number of combination 
result['COMBINATION'] = result['INSULIN'] + result['METFORMIN'] + result['GLP1'] + result['SGLT2']

# medication class to categorize medications (what medications are used)
result['MEDCLASS'] = result['INSULIN'].astype('str') + result['METFORMIN'].astype('str') + result['GLP1'].astype('str') + result['SGLT2'].astype('str')

# medication number
result['MEDNUM'] = result.groupby(by=['PATIENT']).cumcount() + 1

In [157]:
# get previous medication
result['PREVMED'] = result.groupby(by=['PATIENT'])['MEDCLASS'].shift(1, fill_value='0000')

In [158]:
result = result.reset_index()

In [159]:
# rename columns
result = result.rename(
    columns={
        'START':'MEDSTART'
    }
)

In [160]:
result.to_csv('diabetes/m.csv', index=False)

### Add more data : first medication date, previous medication duration

In [161]:
data = pd.read_csv('diabetes/m.csv', converters={'MEDCLASS': lambda x: str(x), 'PREVMED': lambda x: str(x)})

data['PREVMEDSTART'] = data.groupby(by=['PATIENT'])['MEDSTART'].shift(1)

data['MEDSTART'] = pd.to_datetime(data['MEDSTART'], format='%Y-%m-%d')
data['PREVMEDSTART'] = pd.to_datetime(data['PREVMEDSTART'], format='%Y-%m-%d')

data['PREVMEDDUR'] = (data['MEDSTART'] - data['PREVMEDSTART']).dt.days
data['PREVMEDDUR'].fillna(0, inplace=True)

data.head()

data.to_csv('diabetes/m.csv', index=False)

,PATIENT,MEDSTART,ENCOUNTER,INSULIN,METFORMIN,GLP1,SGLT2,COMBINATION,MEDCLASS,MEDNUM,PREVMED,PREVMEDSTART,PREVMEDDUR
0,000177c6-f76b-432b-9493-5a88bc9fb6bd,1957-02-27,351e39d2-52a7-4ce7-928b-5920d9761787,0,1,1,0,2,0110,1,0000,NaT,0.0
1,000177c6-f76b-432b-9493-5a88bc9fb6bd,1995-03-13,351e39d2-52a7-4ce7-928b-5920d9761787,0,1,1,0,2,0110,2,0110,1957-02-27,13893.0
2,000177c6-f76b-432b-9493-5a88bc9fb6bd,1997-03-28,351e39d2-52a7-4ce7-928b-5920d9761787,1,1,1,0,3,1110,3,0110,1995-03-13,746.0
3,00018e41-aba7-4cb3-aeed-fd2d7bcbc6a7,1964-07-10,7f89d06b-9fcf-432c-aa18-defed8f07c16,0,1,0,0,1,0100,1,0000,NaT,0.0
4,00018e41-aba7-4cb3-aeed-fd2d7bcbc6a7,1970-02-09,7f89d06b-9fcf-432c-aa18-defed8f07c16,1,1,0,0,2,1100,2,0100,1964-07-10,2040.0


### 2. Observations
- file : "diabetes/o.csv"
- fields : DATE, PATIENT, ENCOUNTER, HBA1C (%), BMI (kg/m2)

In [163]:
observations = pd.read_csv('set_full/observations.csv')
observations = observations[(observations['CODE'] == Observation.HBA1C.value) | (observations['CODE'] == Observation.BMI.value)]
observations.drop(['DESCRIPTION', 'UNITS'], axis=1, inplace=True, errors='ignore')

# minimum observation value for Diabetes is 0
observations['VALUE'].replace(observations[observations['VALUE'] < 0]['VALUE'].array, np.nan, inplace=True)
observations = observations[~observations['VALUE'].isna()].reset_index(drop=True)
observations
observations.to_csv('diabetes/o.csv', index=False)

In [164]:
observations.tail()

,DATE,PATIENT,ENCOUNTER,CODE,VALUE
4726350,2012-04-15,d503d3e7-fece-4598-8afe-6420b56068ea,10a9887b-5ead-4ac3-81c2-26993eb42726,39156-5,30.91
4726351,2013-05-28,d503d3e7-fece-4598-8afe-6420b56068ea,c3ae2920-c466-4fe0-b642-7e8b9d3057e4,39156-5,30.58
4726352,2014-07-21,d503d3e7-fece-4598-8afe-6420b56068ea,253095a2-36fb-4e3f-ae4c-c7e9d149443d,39156-5,29.81
4726353,2015-08-09,d503d3e7-fece-4598-8afe-6420b56068ea,8d89df20-7bbb-4ed3-ab9d-d4e904a147fc,39156-5,29.33
4726354,2016-08-14,d503d3e7-fece-4598-8afe-6420b56068ea,e584fb72-3860-4bf3-890f-2a504fd4e4d7,39156-5,28.89


### 3. Patient - Medication

- file : "diabetes/p_m.csv"
- fields :
    - [RACE, ETHNICITY, GENDER, PATIENT, ENCOUNTER] 
    - [MEDSTART, INSULIN, METFORMIN, GLP1, SGLT2, DEAD, COMBINATION]
    - [AGEDEAD, AGEMEDICATION, AGEFIRSTMEDICATION, DAYSLIVEFIRSTMED]


In [165]:
# load data
patients = pd.read_csv('set_full/patients.csv')

medications = pd.read_csv('diabetes/m.csv', converters={'MEDCLASS': lambda x: str(x), 'PREVMED': lambda x: str(x)})

# MERGE : PATIENT + MEDICATION

# diabetes patients based on medications record
diabetes = medications
diabetes_patient_ids = diabetes['PATIENT'].unique()

# list of patients who have diabetes
diabetes_patients = patients[patients['ID'].isin(diabetes_patient_ids)]

# data of patient with diabetes medication
merged = diabetes_patients.merge(diabetes, left_on='ID', right_on='PATIENT', how='left')

# Convert data types
merged['BIRTHDATE'] = pd.to_datetime(merged['BIRTHDATE'], format='%d/%m/%y')
merged['DEATHDATE'] = pd.to_datetime(merged['DEATHDATE'], format='%d/%m/%y')
merged['MEDSTART'] = pd.to_datetime(merged['MEDSTART'], format='%Y-%m-%d')

# Fix 2 digits of year misinterpretation
merged.loc[merged['BIRTHDATE'] > merged['MEDSTART'], 'BIRTHDATE'] = merged['BIRTHDATE'] - pd.offsets.DateOffset(years=100)
merged.loc[merged['DEATHDATE'] > pd.to_datetime('2020-12-31'), 'DEATHDATE'] = merged['DEATHDATE'] - pd.offsets.DateOffset(years=100)

# Add additional information

# dead status
merged['DEAD'] =  np.where(merged['DEATHDATE'].isnull(), 0, 1)

# age when dead (accuracy up to month level)
merged['AGEDEAD'] = merged['DEATHDATE'].dt.year - merged['BIRTHDATE'].dt.year - (merged['DEATHDATE'].dt.month < merged['BIRTHDATE'].dt.month)

# when the medication happened
merged['AGEMEDICATION'] = (merged['MEDSTART']).dt.year - (merged['BIRTHDATE']).dt.year - (merged['MEDSTART'].dt.month < merged['BIRTHDATE'].dt.month)

# age first medication
group = merged.groupby(['PATIENT'])
group_age_medication = group['AGEMEDICATION'].min()
temp = pd.DataFrame({'PATIENTID': group_age_medication.keys(), 'AGEFIRSTMEDICATION':group_age_medication.values})
merged = merged.merge(temp, left_on='ID', right_on='PATIENTID', how='left')

# livespan after first medication
merged['DAYSLIVEFIRSTMED'] = (merged['DEATHDATE'] - merged['MEDSTART']).dt.days

# Drop unused columns
merged.drop(['ID', 'BIRTHDATE', 'DEATHDATE', 'PATIENTID'], axis=1, inplace=True, errors='ignore')

merged = merged.drop_duplicates()

# get the patient_medication table
merged = merged.sort_values(by=['PATIENT', 'ENCOUNTER', 'MEDSTART'])

# save
merged.to_csv('diabetes/p_m.csv', index=False)

In [166]:
merged

,RACE,ETHNICITY,GENDER,PATIENT,MEDSTART,ENCOUNTER,INSULIN,METFORMIN,GLP1,SGLT2,...,MEDCLASS,MEDNUM,PREVMED,PREVMEDSTART,PREVMEDDUR,DEAD,AGEDEAD,AGEMEDICATION,AGEFIRSTMEDICATION,DAYSLIVEFIRSTMED
0,white,irish,M,000177c6-f76b-432b-9493-5a88bc9fb6bd,1957-02-27,351e39d2-52a7-4ce7-928b-5920d9761787,0,1,1,0,...,0110,1,0000,NaN,0.0,1,81.0,37,37,16165.0
1,white,irish,M,000177c6-f76b-432b-9493-5a88bc9fb6bd,1995-03-13,351e39d2-52a7-4ce7-928b-5920d9761787,0,1,1,0,...,0110,2,0110,1957-02-27,13893.0,1,81.0,75,37,2272.0
2,white,irish,M,000177c6-f76b-432b-9493-5a88bc9fb6bd,1997-03-28,351e39d2-52a7-4ce7-928b-5920d9761787,1,1,1,0,...,1110,3,0110,1995-03-13,746.0,1,81.0,77,37,1526.0
59623,black,dominican,F,00026c2e-b6f6-4afd-a6f2-10f5b3c609d6,2006-05-22,3ac0e86e-b1fb-404b-8185-fd8cda19d81b,0,1,0,0,...,0100,1,0000,NaN,0.0,0,NaN,41,41,NaN
59624,black,dominican,F,00026c2e-b6f6-4afd-a6f2-10f5b3c609d6,2015-07-09,a003c316-dc05-44fa-8bfb-7ab50d944f74,0,1,1,0,...,0110,2,0100,2006-05-22,3335.0,0,NaN,50,41,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42565,white,italian,M,fffda49e-7bd3-4d38-a497-0165d128010e,1993-05-29,8e65dcd9-1e64-4c7f-be28-13e059fffc47,0,1,0,0,...,0100,1,0000,NaN,0.0,1,66.0,46,46,7346.0
32404,white,french,M,ffff24a2-d219-4aca-aab4-2e45a01632af,1995-09-27,dd6b92f3-0b9f-449b-8dcb-83e137807ede,1,0,0,0,...,1000,1,0000,NaN,0.0,0,NaN,33,33,NaN
39675,white,italian,F,ffff2e14-b1e8-4a73-a79b-90dff97c8116,2016-02-29,03817b40-1bcd-4f49-93c6-2e6dad36ae91,0,1,0,0,...,0100,1,0000,NaN,0.0,0,NaN,44,44,NaN
39676,white,italian,F,ffff2e14-b1e8-4a73-a79b-90dff97c8116,2016-09-03,5c659024-a8c0-4d14-8118-920fed2ff618,1,1,0,0,...,1100,2,0100,2016-02-29,187.0,0,NaN,44,44,NaN


### 4. Medication - Observation

- file : "diabetes/m_o.csv"
- fields : PATIENT, ENCOUNTER, MEDSTART, INSULIN, METFORMIN, GLP1, SGLT2, COMBINATION, OBSDATE, HBA1C, BMI

Assumption : <= useless, because the observations are always in the same date with medications
- The medication given cannot only based on the observation at the same date.
- In this case, I make an assumption that the medication is given based on the last observation within max 1 month



In [167]:
# load medications

medications = pd.read_csv('diabetes/m.csv', converters={'MEDCLASS': lambda x: str(x)})

# load observations
patients = list(set(medications['PATIENT']))

observations = pd.read_csv('diabetes/o.csv')
observations = observations.rename(columns={'DATE':'OBSDATE', 'VALUE': 'OBSVALUE'})

# get obs that related to the medications
observations = observations[observations['PATIENT'].isin(patients)].reset_index(drop=True)

# HBA1C
hba1c = observations[observations['CODE'] == Observation.HBA1C.value]

# MERGE : MEDICATION + OBSERVATIONS (HBA1C)
merged = medications.merge(
    hba1c, 
    left_on=['PATIENT', 'ENCOUNTER', 'MEDSTART'],
    right_on=['PATIENT', 'ENCOUNTER', 'OBSDATE'],
    how='left')
merged = merged[~merged['OBSVALUE'].isna()].reset_index(drop=True)
merged = merged.drop('CODE', axis=1)
merged = merged.rename(columns={'OBSVALUE':'HBA1C'})

# BMI
bmi = observations[observations['CODE'] == Observation.BMI.value]
merged = merged.merge(
    bmi, 
    left_on=['PATIENT', 'ENCOUNTER', 'MEDSTART'],
    right_on=['PATIENT', 'ENCOUNTER', 'OBSDATE'],
    how='left')
merged = merged.drop(['CODE', 'OBSDATE_y'], axis=1)
merged = merged.rename(columns={'OBSVALUE':'BMI', 'OBSDATE_x':'OBSDATE'})

merged
merged.to_csv('diabetes/m_o.csv', index=False)

### 4. Patient - Medication - Observation

- file : "diabetes/p_m_o.csv"
- fields : 
    - [RACE, ETHNICITY, GENDER, PATIENT, ENCOUNTER] 
    - [MEDSTART, INSULIN, METFORMIN, GLP1, SGLT2]
    - [DEAD, AGEDEAD, AGEMEDICATION, AGEFIRSTMEDICATION, DAYSLIVEFIRSTMED, ISFIRST]
    - [OBSDATE, HBA1C, BMI]

In [168]:
pm = pd.read_csv('diabetes/p_m.csv', converters={'MEDCLASS': lambda x: str(x)})

# load observations
observations = pd.read_csv('diabetes/o.csv')
observations = observations.rename(columns={'DATE':'OBSDATE'})

# HBA1C
hba1c = observations[observations['CODE'] == Observation.HBA1C.value]
pmo = pm.merge(
    hba1c, 
    left_on=['PATIENT', 'ENCOUNTER', 'MEDSTART'],
    right_on=['PATIENT', 'ENCOUNTER', 'OBSDATE'],
    how='left')
pmo = pmo[~pmo['VALUE'].isna()].reset_index(drop=True)
pmo = pmo.drop('CODE', axis=1)
pmo = pmo.rename(columns={'VALUE':'HBA1C'})

# BMI
bmi = observations[observations['CODE'] == Observation.BMI.value]
pmo = pmo.merge(
    bmi, 
    left_on=['PATIENT', 'ENCOUNTER', 'MEDSTART'],
    right_on=['PATIENT', 'ENCOUNTER', 'OBSDATE'],
    how='left')
pmo = pmo.drop(['CODE', 'OBSDATE_y'], axis=1)
pmo = pmo.rename(columns={'VALUE':'BMI', 'OBSDATE_x':'OBSDATE'})

pmo.head(5)

pmo.to_csv('diabetes/p_m_o.csv', index=False)

## Goal

1. livespan after the first medication using specific drug 
2. on what condition the medication given? (hba1c level)
3. 

- months do you spent from the first diagnosis until dead
- years until another medication? or another encounter?
- from diagnosis to first medication? then to second medication?
- proportion of patients get particular medication (metformin)

- likelihood for new patient using particular drug
- looking at diabetes + another disease


In [ ]:
df = pd.DataFrame(np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]]), columns=['a', 'b', 'c'])
df['x1'] = df['a'].shift(1, fill_value=0)
df